# Tutorial for onlineSG

this is a Jupyter notebook, it's like a console but with cells that can be run with the play button above or with Ctrl+Enter. Run cells in a sequential way to follow the tutorial in the correct way.

## Game and Player

Game and Player are the two fundamental classes of onlineSG.

The Game class represent a game from Game Theory.

In [2]:
import sys

In [ ]:
sys.path.append("../")

Game takes as arguments the *values* and the *time_horizon*. They are called values and not payoffs, because they represent the value that defenders/attackers give to a target. Payoffs are computed from the values with this convention:

For attackers, for each attacked target:
- covered target payoffs: *0* 
- uncovered target payoff: *attacker value*

For defenders, for each attacked target:
- covered target payoffs: *0* 
- uncovered target payoff: *-defender value*

Let's see an example of game:

In [35]:
from source.game import Game
from source.player import *

In [42]:
values = ((1,1),(2,2),(3,3))
time_horizon = 5
g = Game(values, time_horizon)
defender = Defender(g, 0, 1)
attacker = StackelbergAttacker(g, 1, 1)
g.set_players([defender], [attacker])

each game has some players that can be either attackers or defenders. 

This is a simple example in which the defender is a Stochastic player and his adversary is a Stackelberg player

Now let's see a game round

In [43]:
g.strategy_history.append(dict()) # initialize the strategy history of this round
g.strategy_history[-1][defender.id] = defender.compute_strategy() # defender compute his strategy
g.strategy_history[-1][attacker.id] = attacker.compute_strategy() # attacker possibly observe and compute his strategy
g.history.append(dict()) # initialize the history of this round
for p in g.players: # each player sample a realization of its strategy
    g.history[-1][p] = g.players[p].sample_strategy()

Each game has a strategy_history attribute

In [45]:
g.strategy_history

[{0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]}]

and a history attribute

In [44]:
g.history

[{0: [2], 1: [2]}]

This procedure is complex and takes into account the possibility of multiple defenders. In most of the cases we have to deal with only a defender and in general multiple attacker. Therefore it is useful to make an abstraction layer in which we only have one *agent* and the his adversaries are modeled as the *environment*.

In [48]:
from source.environment import Environment

In [62]:
values = ((1, 1), (2, 2), (3, 3))
time_horizon = 5
g = Game(values, time_horizon)
agent = Defender(g, 0, 1)
attacker = StackelbergAttacker(g, 1, 1)
g.set_players([agent], [attacker])
e = Environment(g, 0)

then the interaction becomes

In [63]:
strategy = agent.compute_strategy()
e.observe_strategy(strategy)
realization = agent.sample_strategy()
e.observe_realization(realization)
feedback = e.feedback("expert")
agent.receive_feedback(feedback)

In [64]:
g.history

[{0: [1], 1: [2]}]

to run the game till the end then:

In [65]:
for t in range(g.time_horizon):
    strategy = agent.compute_strategy()
    e.observe_strategy(strategy)
    realization = agent.sample_strategy()
    e.observe_realization(realization)
    feedback = e.feedback("expert")
    agent.receive_feedback(feedback)

and this is what happened

In [66]:
g.strategy_history

[{0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]}]

In [67]:
g.history

[{0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [2], 1: [2]},
 {0: [2], 1: [2]},
 {0: [2], 1: [2]}]

In [68]:
agent.feedbacks

[{0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: 0},
 {0: 0, 1: 0, 2: 0},
 {0: 0, 1: 0, 2: 0}]

## Experiment
In order to run interactions in an easier way we can use the class Experiment as a wrapper

In [69]:
from source.runner import Experiment

In [87]:
values = ((1, 1), (2, 2), (3, 3))
time_horizon = 5
g = Game(values, time_horizon)
agent = Defender(g, 0, 1)
attacker = StackelbergAttacker(g, 1, 1)
g.set_players([agent], [attacker])
experiment = Experiment(g)
experiment.run()

In [88]:
experiment.game.history

[{0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [2], 1: [2]},
 {0: [2], 1: [2]},
 {0: [0], 1: [2]}]

In [89]:
experiment.agent

<Defender id:0 resources:1>

In [90]:
experiment.environment

<Environment>

The *Experiment* class assumes that there is only one defender (the first player) and computes the Environment with the remaining players.

The experiment class handles also the seed initialization of the pseudo-random number generation:

In [91]:
experiment.seed

0.8631987818666238

if no seed is given as argument then it automatically initialize it, otherwise it is possible to pass it to the object, for example to run again an experiment:

In [92]:
# different random seed from before
values = ((1, 1), (2, 2), (3, 3))
time_horizon = 5
g = Game(values, time_horizon)
agent = Defender(g, 0, 1)
attacker = StackelbergAttacker(g, 1, 1)
g.set_players([agent], [attacker])
experiment2 = Experiment(g)
experiment2.run()
experiment2.game.history

[{0: [1], 1: [2]},
 {0: [2], 1: [2]},
 {0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [0], 1: [2]}]

In [96]:
experiment2.seed

0.346289571348973

In [94]:
# same seed from the first case, which give the same results
values = ((1, 1), (2, 2), (3, 3))
time_horizon = 5
g = Game(values, time_horizon)
agent = Defender(g, 0, 1)
attacker = StackelbergAttacker(g, 1, 1)
g.set_players([agent], [attacker])
experiment3 = Experiment(g, experiment.seed)
experiment3.run()
experiment3.game.history

[{0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [2], 1: [2]},
 {0: [2], 1: [2]},
 {0: [0], 1: [2]}]

In [95]:
experiment3.seed

0.8631987818666238

## Run games from configurations files
Configuration files can be created to load many games at the same time. Configuration files are csv file with a format like this:

In [135]:
with open("../games/conf.csv", "r") as f:
    print(f.read())

T,0,1,2,3,4,Defender,Attacker,Attacker
5,(1 7 1),(1 1 12),(12 1 1),(1 1 2),(2 2 3),defender,player,stackelberg
3,1,1,1,1,1,defender,,stackelberg
6,1,3,1,2,1,stu_defender,stackelberg,
5,1,12,13,1,1,defender,player,stackelberg



the first row is the header in which we have three groups of column types:
- time horizon (1 column)
- target values (many)
- Defender (in general many, for now 1 column)
- Attacker (many) 

Attackers headers represent the maximum number of attackers, but the relative cells could also be void

In [77]:
from source.runner import Batch

The *Batch* object use the *Parser* object to parse each row and then produce a related *Configuration* object.

In [102]:
conf = "../games/conf.csv"
b = Batch(conf, ".", print_results=False)
b.parse_batch()

In [80]:
len(b.configurations) # configurations obtained from the file

4

In [81]:
b.configurations[0] # the first configuration

<Configuration game:<Game values:[[1, 7, 1], [1, 1, 12], [12, 1, 1], [1, 1, 2], [2, 2, 3]] players{0: <Defender id:0 resources:1>, 1: <Attacker id:1 resources:1>, 2: <StackelbergAttacker id:2 resources:1>} time_horizon:5> experiments:[]>

The *Configuration* object contains the initial setting of the game and the relative players, and it is able to run and store multiple experiments with different seeds.

In [103]:
c = b.configurations[2]

In [104]:
c.run_an_experiment()
c.run_an_experiment()
c.run_an_experiment()

In [105]:
c.experiments

[<Experiment seed:0.037158469196690413>,
 <Experiment seed:0.026354299819451588>,
 <Experiment seed:0.6106033959583529>]

In order to run once all the configurations of a batch we can use:

In [106]:
b.run()

In [110]:
for i,c in enumerate(b.configurations):
    print(i, ": ", c.experiments)

0 :  [<Experiment seed:0.06882945117146566>]
1 :  [<Experiment seed:0.23790797623136462>]
2 :  [<Experiment seed:0.037158469196690413>, <Experiment seed:0.026354299819451588>, <Experiment seed:0.6106033959583529>, <Experiment seed:0.16881640098522077>]
3 :  [<Experiment seed:0.6749084098352461>]


### WARNING:
The seed initialization mechanism is guaranteed to generate identical experiments provide that they are run in a **SEQUENTIAL** way: in fact there could be problem with parallelization

## Run configuration files from a folder and save output

In [112]:
from source.runner import Runner

Use the class Runner to load all the configuration files of a folder in the respective Batch objects. It takes as arguments the path of the folder and the location where you want to store the results (it should not be an already existing folder!).

In [114]:
mypath = "../games/"
resultspath = "results"
r = Runner(mypath, resultspath)
r.run()

In [116]:
len(r.batches) # loaded batches

1

In [128]:
import os

In [131]:
os.listdir() # let's see what have been saved

['conf',
 '.ipynb_checkpoints',
 'prova_json',
 'Stackerlberg Best Response.ipynb',
 'Linear Programming Examples.ipynb',
 'Tutorial.ipynb',
 'results',
 'prova',
 'Example.ipynb',
 'gurobi.log']

a folder is generated for each configuration file in the folder

In [132]:
os.listdir("results")

['conf']

a folder is generated for each row of the file

In [133]:
os.listdir("results/conf")

['1', '2', 'batch.csv', '3', '0']

each row corresponds to a configuration and in its folder there are:
- 'seeds.txt': a file with the used seeds
- 'json.txt': a file with a json of the Configuration
- 'game': a binary pickle file with the Configuration object
- #seed csv files with the output of the relative experiments

let's see in detail each file

In [158]:
os.listdir("results/conf/1")

['seeds.txt', '0.43507662667582414', 'json.txt', 'game']

In [194]:
# seed.txt
with open("results/conf/1/seeds.txt", "r") as f:
    seed = f.read().rstrip()
    print(seed)

0.43507662667582414


In [160]:
# json.txt
with open("results/conf/1/json.txt", "r") as f:
    print(f.read())

{
    "attackers": [
        1
    ],
    "defenders": [
        0
    ],
    "history": [],
    "players": {
        "0": {
            "class_name": "Defender",
            "feedbacks": [],
            "id": 0,
            "resources": 1
        },
        "1": {
            "class_name": "StackelbergAttacker",
            "id": 1,
            "resources": 1
        }
    },
    "strategy_history": [],
    "time_horizon": 3,
    "values": [
        [
            1.0,
            1.0
        ],
        [
            1.0,
            1.0
        ],
        [
            1.0,
            1.0
        ],
        [
            1.0,
            1.0
        ],
        [
            1.0,
            1.0
        ]
    ]
}


In [195]:
# 0.8865257094040085 csv file 
import pandas as pd

# for each player column we have a pair (strategy-realization)
pd.read_csv("results/conf/1/"+seed)

,Unnamed: 0,defender-0,stackelberg-1,feedback target 0,feedback target 1,feedback target 2,feedback target 3,feedback target 4
0,0,"([0.2, 0.2, 0.2, 0.2, 0.2], [3])","([1, 0, 0, 0, 0], [0])",-1,-0,-0,-0,-0
1,1,"([0.2, 0.2, 0.2, 0.2, 0.2], [3])","([1, 0, 0, 0, 0], [0])",-1,-0,-0,-0,-0
2,2,"([0.2, 0.2, 0.2, 0.2, 0.2], [2])","([1, 0, 0, 0, 0], [0])",-1,-0,-0,-0,-0


Finally it is possible to recover the initial game in python using the serialized file 'game'

In [146]:
from source.game import load

In [263]:
game = load("results/conf/1/game")
game

<Game values:[[1.0, 1.0], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0]] players{0: <Defender id:0 resources:1>, 1: <StackelbergAttacker id:1 resources:1>} time_horizon:3>

In [264]:
game.history # the game is saved before it is played

[]

if we want we can try it with the same seed as before:

In [265]:
with open("results/conf/1/seeds.txt", "r") as f:
    seed = float(f.read().rstrip())

In [266]:
experiment = Experiment(game, seed)
experiment.run()

In [267]:
experiment.game.history

[{0: [3], 1: [0]}, {0: [3], 1: [0]}, {0: [2], 1: [0]}]

In [268]:
experiment.seed

0.43507662667582414

or with another one

In [269]:
game = load("results/conf/1/game") # reload the game
experiment = Experiment(game)
experiment.run()

In [270]:
experiment.game.history

[{0: [0], 1: [0]}, {0: [4], 1: [0]}, {0: [3], 1: [0]}]

In [271]:
experiment.seed

0.5104161566816114